In [ ]:
import psycopg2
import pandas as pds
from sqlalchemy import create_engine
import datetime
from typing import List, Tuple
import random
from collections import defaultdict

## Extracción de los datos sobre las tablas

In [ ]:
DATABASE = "proyecto"
PORT="5432"
HOST="localhost"
USERNAME="alexndf"
PASSWORD="postgrespw123"

alchemy_engine   = create_engine(f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}')
db_connection = alchemy_engine.connect()

table_data_query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_type = 'BASE TABLE'
        AND table_schema = 'public'
"""
table_data = pds.read_sql(table_names_query, db_connection)

table_column_data_query = """
    SELECT c.table_name,
        c.column_name
    FROM pg_catalog.pg_statio_all_tables AS st
        INNER JOIN pg_catalog.pg_description pgd ON (pgd.objoid = st.relid)
        INNER JOIN information_schema.columns c ON (
            pgd.objsubid = c.ordinal_position
            AND c.table_schema = st.schemaname
            AND c.table_name = st.relname
        );
"""
table_column_data = pds.read_sql(table_column_data_query, db_connection)

In [ ]:
display(table_data)
display(table_column_data)

In [ ]:
def get_table_primary_keys(table_name):
    pk_query = f"""
        SELECT c.column_name
        FROM information_schema.table_constraints tc 
            JOIN information_schema.constraint_column_usage AS ccu USING (constraint_schema, constraint_name) 
            JOIN information_schema.columns AS c ON c.table_schema = tc.constraint_schema
                AND tc.table_name = c.table_name AND ccu.column_name = c.column_name
            WHERE constraint_type = 'PRIMARY KEY' AND c.table_name='{table_name}';
    """
    return pds.read_sql(pk_query, db_connection)

In [ ]:
primary_keys = defaultdict(lambda : set())
other_columns = defaultdict(lambda : set())

for tn in table_data["table_name"]:
    for pk in get_table_primary_keys(tn)["column_name"]:
        primary_keys[tn].add(pk)
    for col in table_column_data[table_column_data["table_name"]==tn]["column_name"]:
        if col not in primary_keys[tn]:
            other_columns[tn].add(col)

In [ ]:
db_connection.close()

## Generación de las dependencias funcionales

In [ ]:
def format_tex_col(col_name: str):
    return r"\texttt{" + col_name.replace("_", r"\_") + "}"


In [ ]:
for tn in table_data["table_name"]:
    pks = primary_keys[tn]
    cols = other_columns[tn]
    print(tn)
    print("\tprimary_keys:", pks)
    print("\tother_cols:", cols)
    print("")

In [ ]:
functional_dependencies_latex_strs = []
for tn in sorted(table_data["table_name"]):
    pks = primary_keys[tn]
    cols = other_columns[tn]
    if len(pks) == 0 or len(cols) == 0:
        continue
    pk_str = ", ".join(map(format_tex_col, [tn + "." + pk for pk in sorted(pks)]))
    cols_str = (
        "\t\\begin{itemize}"
        + "\n"
        + "\n".join(["\t\t\\item " + format_tex_col(tn + "." + c) for c in sorted(cols)])
        + "\n"
        + "\t\\end{itemize}"
    )
    if pk_str != cols_str:
        fd_str = "\t" + r"\item " + pk_str + " $\\to$ \n" + cols_str
        functional_dependencies_latex_strs.append(fd_str)


In [ ]:
fd_tex_str = (
    r"\begin{itemize}"
    + "\n"
    + "\n".join(functional_dependencies_latex_strs)
    + "\n"
    + r"\end{itemize}"
)
print(fd_tex_str)
